## Data Loaders

This notebook demonstrates the use of **vertex loader** in `pyTigerGraph`. The job of a data loader is to pull data from the TigerGraph database. Currently, the following data loaders are provided:
* EdgeLoader, which returns batches of edges.
* VertexLoader, which returns batches of vertices.
* GraphLoader, which returns randomly sampled (probably disconnected) subgraphs in pandas `dataframe`, `PyG` or `DGL` format.
* NeighborLoader, which returns subgraphs using neighbor sampling in `dataframe`, `PyG` or `DGL` format.
* EdgeNeighborLoader, which returns subgraphs using neighbor sampling from edges in `dataframe`, `PyG` or `DGL` format.

Every data loader above can either get all the batches as a HTTP response (default) or stream every batch through Kafka. The former mechanism is good for testing with small graphs and it is fast, but it subjects to a data size limit of 2GB. For large graphs, the HTTP channel will likely fail due to size limit and network connectivity issues. Streaming via Kafka is offered for data robustness and scalability. Also, Kafka excels at multi-consumer use cases, and it is efficient for model search or hyperparameter tuning when there are multiiple consumers of the same data. 

The data loaders support both homogeneous and heterogenous graphs. By default, they load from all vertex and edge types and treat the graph as a homogeneous graph. But they also allow users to specify what vertex and edge types to load from and what attributes to load from each type. This way users will get heterogeneous graph outputs.


### Connection to Database

The `TigerGraphConnection` class represents a connection to the TigerGraph database. Under the hood, it stores the necessary information to communicate with the database. It is able to perform quite a few database tasks. Please see its [documentation](https://docs.tigergraph.com/pytigergraph/current/intro/) for details.

In [1]:
from pyTigerGraph import TigerGraphConnection
import json

# Read in DB configs
with open('../../config.json', "r") as config_file:
    config = json.load(config_file)
    
conn = TigerGraphConnection(
    host=config["host"],
    username=config["username"],
    password=config["password"]
)

from pyTigerGraph.datasets import Datasets

dataset = Datasets("Cora")

conn.ingestDataset(dataset, getToken=config["getToken"])

from pyTigerGraph.visualization import drawSchema

drawSchema(conn.getSchema(force=True))

A folder with name Cora already exists in ./tmp. Skip downloading.
---- Checking database ----
A graph with name Cora already exists in the database. Skip ingestion.
Graph name is set to Cora for this connection.


CytoscapeWidget(cytoscape_layout={'name': 'circle', 'animate': True, 'padding': 1}, cytoscape_style=[{'selecto…

<span style="color:red">Uncomment cell below and run to get and set token if token authentication is enabled</span>. 
* This is required for all databases on tgcloud.
* `<secret>` is your user secret. See https://docs.tigergraph.com/tigergraph-server/current/user-access/managing-credentials#_secrets for details.
* If you don't know your secret, you can use `secret=conn.createSecret()` to create one.

### Vertex Loader

`VertexLoader` pulls batches of vertices from database. Specifically, it divides vertices into `num_batches` and returns each batch separately. The boolean attribute provided to `filter_by` indicates which vertices are included. If you need random batches, set `shuffle` to True.

**Note**: For the first time you initialize the loader on a graph in TigerGraph, the initialization might take a minute as it installs the corresponding query to the database and optimizes it. However, the query installation only needs to be done once, so it will take no time when you initialize the loader on the same TG graph again.

There are two ways to use the data loader. See [here](https://github.com/TigerGraph-DevLabs/mlworkbench-docs/blob/main/tutorials/basics/2_dataloaders.ipynb) for examples.
* First, it can be used as an iterable, which means you can loop through it to get every batch of data. If you load all vertices at once (`num_batches=1`), there will be only one batch (of all the vertices) in the iterator.
* Second, you can access the `data` property of the class directly. If there is only one batch of data to load, it will give you the batch directly instead of an iterator, which might make more sense in that case. If there are multiple batches of data to load, it will return the loader again.

Args:
* attributes (list or dict, optional):
                Vertex attributes to be included. If it is a list, then the attributes
                in the list from all vertex types will be selected. An error will be thrown if
                certain attribute doesn't exist in all vertex types. If it is a dict, keys of the 
                dict are vertex types to be selected, and values are lists of attributes to be 
                selected for each vertex type. Defaults to None.
* batch_size (int, optional):
                Number of vertices in each batch.
                Defaults to None.
* num_batches (int, optional):
                Number of batches to split the vertices.
                Defaults to 1.
* shuffle (bool, optional):
                Whether to shuffle the vertices before loading data.
                Defaults to False.
* filter_by (str, optional):
                A boolean attribute used to indicate which vertices
                can be included. Defaults to None.
* output_format (str, optional):
                Format of the output data of the loader. Only
                "dataframe" is supported. Defaults to "dataframe".
* loader_id (str, optional):
                An identifier of the loader which can be any string. It is
                also used as the Kafka topic name. If `None`, a random string will be generated
                for it. Defaults to None.
* buffer_size (int, optional):
                Number of data batches to prefetch and store in memory. Defaults to 4.
* timeout (int, optional):
                Timeout value for GSQL queries, in ms. Defaults to 300000.

# Testcase1: using vertexLoader with callback_fn to get batches of edges.(for homogeneous graph)  
## Results: run successfully, data loaded completely

In [2]:
def process_batch(batch):
    return batch
vertex_loader1 = conn.gds.vertexLoader(
    num_batches=10, 
    attributes=["id", "x", "y"],
    callback_fn = process_batch
)
for i, batch in enumerate(vertex_loader1):
    print("----Batch {}----".format(i))
    print(batch.shape)
    print(batch.head())

----Batch 0----
(2708, 4)
         vid    id                                                  x  y
0  100663296  2703  0 0 0 0 0 1 0 0 0 0 0 0 0 0 0 0 0 0 0 1 0 0 0 ...  3
1  100663297  2508  0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 ...  3
2  100663298  2379  0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 1 0 0 0 ...  4
3  100663299  2365  0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 ...  2
4  100663300  2277  0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 ...  0


# Testcase2: using vertexLoader with callback_fn to get batchs of vertices(for heterogeneous graph).  
## case details:using vertexLoader without callback_fn first, then using vertexLoader with callback_fn to get part of data.
## Results: run successfully, data loaded completely

Since `Cora` is a homogeneous graph, we will connect to a different graph to demostrate the use case of heterogeneous graphs.

In [4]:
print(conn.gsql("CREATE GRAPH hetero()"))

conn.graphname="hetero"
# Create graph schema
print(conn.gsql(open("./data/heterogenous/gsql/schema.gsql", "r").read()))
# Create loading job
print(conn.gsql(open("./data/heterogenous/gsql/load.gsql", "r").read()))

# COMMENT OUT THE LINE BELOW if you are NOT using a graph that requires token authentication
conn.getToken(conn.createSecret())

# Load data
conn.runLoadingJobWithFile("./data/heterogenous/v0.csv", "v0_csv", "load_hetero_data")
conn.runLoadingJobWithFile("./data/heterogenous/v1.csv", "v1_csv", "load_hetero_data")
conn.runLoadingJobWithFile("./data/heterogenous/v2.csv", "v2_csv", "load_hetero_data")
conn.runLoadingJobWithFile("./data/heterogenous/v0v0.csv", "v0v0_csv", "load_hetero_data")
conn.runLoadingJobWithFile("./data/heterogenous/v1v1.csv", "v1v1_csv", "load_hetero_data")
conn.runLoadingJobWithFile("./data/heterogenous/v1v2.csv", "v1v2_csv", "load_hetero_data")
conn.runLoadingJobWithFile("./data/heterogenous/v2v0.csv", "v2v0_csv", "load_hetero_data")
conn.runLoadingJobWithFile("./data/heterogenous/v2v1.csv", "v2v1_csv", "load_hetero_data")
conn.runLoadingJobWithFile("./data/heterogenous/v2v2.csv", "v2v2_csv", "load_hetero_data")

Semantic Check Fails: The graph name conflicts with another type or existing graph names! Please use a different name.
The graph hetero could not be created!
Using graph 'hetero'
Semantic Check Fails: The vertex name v0 is used by another object! Please use a different name.
Failed to create schema change jobs: [hetero_job].
Using graph 'hetero'
Semantic Check Fails: The job name load_hetero_data already exists in other objects!
Failed to create loading jobs: [load_hetero_data].


[{'sourceFileName': 'Online_POST',
  'statistics': {'validLine': 966,
   'rejectLine': 0,
   'failedConditionLine': 0,
   'notEnoughToken': 0,
   'invalidJson': 0,
   'oversizeToken': 0,
   'vertex': [],
   'edge': [{'typeName': 'v2v2',
     'validObject': 966,
     'noIdFound': 0,
     'invalidAttribute': 0,
     'invalidVertexType': 0,
     'invalidPrimaryId': 0,
     'invalidSecondaryId': 0,
     'incorrectFixedBinaryLength': 0}],
   'deleteVertex': [],
   'deleteEdge': []}}]

In [ ]:
loader3 = conn.gds.vertexLoader(
    attributes={"v0": ["x", "y"],
                "v1": ["x"]},
    batch_size=20,
    shuffle=True,
    filter_by=None,
    loader_id=None,
    buffer_size=4,
)
for i, batch in enumerate(loader3):
    print("----Batch {}----".format(i))
    for j in batch:
        print("Vertex type:", j)
        print(batch[j].head(1))

----Batch 0----
Vertex type: v0
         vid                                                  x  y
0  135266304  -1.39324 -0.815802 0.524335 1.28013 -0.299068 ...  4
Vertex type: v1
         vid                                                  x
0  181403652  1.28591 -1.024 -0.952083 -0.757218 -0.973589 0...
----Batch 1----
Vertex type: v0
         vid                                                  x  y
0  152043520  -1.50195 -0.251676 -0.876361 0.351269 -0.72887...  4
Vertex type: v1
         vid                                                  x
0  185597952  0.280565 0.222271 -1.08299 1.05294 -0.404641 0...
----Batch 2----
Vertex type: v0
         vid                                                  x  y
0  161480704  1.2661 1.56835 0.91763 -1.23061 -0.609033 0.55...  1
Vertex type: v1
         vid                                                  x
0  186646531  1.41948 -0.721332 -0.251678 -1.84578 -0.362447...
----Batch 3----
Vertex type: v0
         vid                          

In [6]:
def process_batch(batch):
    return {"v0":batch["v0"]}
loader4 = conn.gds.vertexLoader(
    attributes={"v0": ["x", "y"],
                "v1": ["x"]},
    batch_size=20,
    shuffle=True,
    filter_by=None,
    loader_id=None,
    buffer_size=4,
    callback_fn=process_batch
)
for i, batch in enumerate(loader4):
    print("----Batch {}----".format(i))
    for j in batch:
        print("Vertex type:", j)
        print(batch[j].head(1))

----Batch 0----
Vertex type: v0
         vid                                                  x  y
0  134217732  0.292398 0.947335 -1.09759 -0.389061 1.1454 -1...  3
----Batch 1----
Vertex type: v0
         vid                                                  x  y
0  134217728  -0.188107 -0.301286 1.2804 0.675936 0.717697 1...  1
----Batch 2----
Vertex type: v0
         vid                                                  x  y
0  138412033  1.96028 -0.881764 -0.00337484 0.277803 -0.3281...  8
----Batch 3----
Vertex type: v0
         vid                                                  x  y
0  138412032  -1.4369 0.245456 0.229138 1.0582 -0.0612754 -0...  5
----Batch 4----
Vertex type: v0
         vid                                                  x  y
0  137363457  -1.00629 -0.767798 0.199938 0.992184 0.736447 ...  7
----Batch 5----
Vertex type: v0
         vid                                                  x  y
0  136314880  -1.29082 0.908381 0.621846 -1.05145 -1.24387 1...  1
----

# Testcase3: using vertexLoader with callback_fn to loaddata(via Kafka).  
## Results: run successfully, data loaded completely

**Note**: Kafka streaming function is only available for the Enterprise Edition. You need to activate the Enterprise Edition to use it. 

In [36]:
conn.graphname="Cora"
# COMMENT OUT THE LINE BELOW if you are NOT using a graph that requires token authentication
conn.getToken(conn.createSecret())

('g3ifmt4jgjk46g478dkdihio4j4d7ppm', 1674807685, '2023-01-27 08:21:25')

#### Configure Kafka
Set up Kafka here. Once configured, the settings will be shared with all newly created data loaders and no need to set up Kafka for each loader. Please see official [doc](https://docs.tigergraph.com/pytigergraph/current/gds/gds#_configurekafka) for detailed settings.

In [37]:
conn.gds.configureKafka(kafka_address ="your_Kafka_address")

#### Get batches of vertices

In [40]:
def process_batch(batch):
    return batch
vertex_loader5 = conn.gds.vertexLoader(
    num_batches=20, 
    attributes=["id", "x", "y"],
    shuffle=True,
    filter_by=None,
    callback_fn = process_batch
)
for i, batch in enumerate(vertex_loader5):
    print("----Batch {}----".format(i))
    print(batch.shape)
    print(batch.head(1))

----Batch 0----
(128, 4)
         vid  id                                                  x  y
0  102760477  16  0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 ...  2
----Batch 1----
(146, 4)
         vid    id                                                  x  y
0  100663320  1295  0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 1 0 0 0 ...  3
----Batch 2----
(147, 4)
         vid    id                                                  x  y
0  118489096  1878  0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 ...  5
----Batch 3----
(123, 4)
         vid    id                                                  x  y
0  119537676  2173  0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 ...  6
----Batch 4----
(124, 4)
         vid   id                                                  x  y
0  100663359  668  0 0 0 0 1 0 0 0 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 ...  5
----Batch 5----
(143, 4)
         vid    id                                                  x  y
0  102760461  2035  0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0